In [1]:
#| default_exp Agents/APOStrategySarsa

In [1]:
#| export
import numpy as np
import itertools as it
from functools import partial

import jax
from jax import jit
import jax.numpy as jnp

from fastcore.utils import *

from pyCRLD.Agents.StrategyBase import strategybase
from pyCRLD.Utils.Helpers import *

In [2]:
#| export   
class stratSARSA(aPObase, strategybase):
    """
    Class for CRLD-SARSA agents in strategy space.
    """
    
    def __init__(self, env, learning_rates, discount_factors,
                 choice_intensities=1, **kwargs):
        """
        Parameters
        ----------
        env : environment object
        learning_rates : the learning rate(s) for the agents
        discount_factors : the discount factor(s) for the agents
        choice_intensities : inverse temperature of softmax / exploitation level
        """
        self.env = env
        Tt = env.T; assert np.allclose(Tt.sum(-1), 1)
        Rt = env.R
        Ot = env.O
        super().__init__(Tt, Rt, Ot, discount_factors, **kwargs)
        assert np.allclose(env.F, 0), 'PO learning w final state not def.'

    
    @partial(jit, static_argnums=(0,2))
    def RPEisa(self,
               Xisa,  # Joint strategy
               norm=False # normalize error around actions? 
               ) -> np.ndarray:  # RP/TD error
        """
        Compute reward-prediction/temporal-difference error for 
        strategy SARSA dynamics, given joint strategy `Xisa`.
        """
        R = self.Risa(Xisa)
        NextQ = self.NextQisa(Xisa, Risa=R)

        n = jnp.newaxis
        E = self.pre[:,n,n]*R + self.gamma[:,n,n]*NextQ - 1/self.beta[:, n, n] * jnp.log(Xisa)
        E *= self.beta[:,n,n]

        E = E - E.mean(axis=2, keepdims=True) if norm else E
        return E
    
    @partial(jit, static_argnums=0)
    def NextQisa(self,
                 Xisa,          # Joint strategy
                 Qisa=None,  # Optional state-action values for speed-up
                 Risa=None,  # Optional rewards for speed-up
                 Vis=None,   # Optional state values for speed-up
                 Tisas=None  # Optional transition for speed-up
                ) -> jnp.ndarray: # Next values       
        """
        Compute strategy-average next state-action value for agent `i`, current state `s` and action `a`.
        """
        Qisa = self.Qisa(Xisa, Risa=None, Vis=None, Tisas=None)\
            if Qisa is None else Qisa
        
        i = 0; a = 1; s = 2; s_ = 3
        j2k = list(range(6, 6+self.N-1))  # other agents
        b2d = list(range(6+self.N-1, 6+self.N-1 + self.N))  # all actions
        e2f = list(range(5+2*self.N, 5+2*self.N + self.N-1))  # all other acts

        sumsis = [[j2k[l], s, e2f[l]] for l in range(self.N-1)]  # sum inds
        otherX = list(it.chain(*zip((self.N-1)*[Xisa], sumsis)))

        NextQis = jnp.einsum(Qisa, [i, s_, a], Xisa, [i, s_, a], [i, s_])
                    
        args = [self.Omega, [i]+j2k+[a]+b2d+e2f] + otherX +\
            [self.T, [s]+b2d+[s_], NextQis, [i, s_], [i, s, a]]                                            
        return jnp.einsum(*args, optimize=self.opti)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()